In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import trend_finder_lr as tflr
import trend_finder_ovr as tfovr
import ml_models as ml
import trend_finder_ovr as ovr
import yfinance as yf
import pickle
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, svm
from imblearn.over_sampling import RandomOverSampler

# Run a test looking for stock price increases and decreases of percent, 30 days out
- Record 60 trading days before the trend
- use those 60 trading days as features

In [3]:
tesla = tflr.Historydf('TSLA', '1y', 30, 5, 60)

In [4]:
tesla.df

,Open,Close,Volume
0,302.359985,296.070007,53230000
1,297.429993,288.089996,57163900
2,282.829987,284.820007,41864700
3,287.869995,277.700012,50541800
4,280.619995,275.609985,52107300
...,...,...,...
247,221.550003,231.279999,135702700
248,240.250000,233.190002,130597900
249,229.339996,236.860001,101077600
250,238.660004,230.039993,99519300


# Find trends, enter [Buy/Sell] classifier, and create new dataframe

In [5]:
tesla.find_trends()

start index = 59, end index = 222


In [6]:
tesla.pretrends_df

,Open,Close,Volume,Buy/Sell
0,231.309998,238.539993,89330349.0,NaN
1,302.359985,296.070007,53230000.0,NaN
2,297.429993,288.089996,57163900.0,NaN
3,282.829987,284.820007,41864700.0,NaN
4,287.869995,277.700012,50541800.0,NaN
...,...,...,...,...
7735,278.429993,274.429993,113602000.0,NaN
7736,276.470001,269.609985,119425400.0,NaN
7737,268.649994,269.790009,91972400.0,NaN
7738,276.329987,271.989990,95672100.0,NaN


# Structure the data in preperation for training model and normalize the values

In [7]:
tesla.structure_data()

In [8]:
tesla.structured_df

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,Buy/Sell
0,0.408579,0.948965,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,0.723228,0.726422,...,0.624819,0.573016,0.957071,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0
1,0.948965,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,0.723228,0.726422,0.788789,...,0.573016,0.957071,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0.247377,0
2,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,0.723228,0.726422,0.788789,0.867661,...,0.957071,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0.247377,0.376319,0
3,0.814164,0.849858,0.798513,0.741572,0.801700,0.742280,0.745255,0.803328,0.876771,0.940864,...,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0.247377,0.376319,0.561635,0
4,0.849858,0.798513,0.741572,0.801700,0.742280,0.745255,0.803328,0.876771,0.940864,0.885482,...,1.000000,0.782224,0.518301,0.507200,0.212940,0.186382,0.325774,0.526109,0.354377,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0.293438,0.233518,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,0.092498,0.055299,...,0.741057,0.618757,0.576444,0.344522,0.189711,0.250105,0.346537,0.255366,0.200575,0
125,0.233518,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,0.092498,0.055299,0.058919,...,0.618757,0.576444,0.344522,0.189711,0.250105,0.346537,0.255366,0.200575,0.247984,0
126,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,0.092498,0.055299,0.058919,0.000000,...,0.576444,0.344522,0.189711,0.250105,0.346537,0.255366,0.200575,0.247984,0.024484,0
127,0.259396,0.265789,0.203789,0.104205,0.093654,0.092498,0.055299,0.058919,0.000000,0.063617,...,0.344522,0.189711,0.250105,0.346537,0.255366,0.200575,0.247984,0.024484,0.054604,0


# Run logistic regression

In [9]:
tesla_ovr = ml.svm_ovr('Tesla')

In [10]:
tesla_ovr.train_model(tesla.structured_df)

0    71
1    71
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.97


# Adding a third classification for hold
- hold is represented by 2
- will include any points without increases or decreases of desired percentage
- model being used SVM

In [11]:
tesla = tfovr.Historydf('TSLA', '5y', 10, 5, 60)

In [12]:
tesla.find_trends()

start index = 59, end index = 1248


In [13]:
tesla.pretrends_df

,Open,Close,Volume,Buy/Sell
0,231.309998,238.654999,89378807.0,NaN
1,21.200001,21.284666,196189500.0,NaN
2,21.227333,20.790667,114736500.0,NaN
3,20.684668,20.334000,111711000.0,NaN
4,20.150667,20.209999,108250500.0,NaN
...,...,...,...,...
71335,260.970001,253.860001,99242600.0,NaN
71336,251.449997,251.449997,111097900.0,NaN
71337,247.449997,249.699997,96642200.0,NaN
71338,250.869995,242.190002,101596300.0,NaN


In [14]:
tesla.structure_data()

In [15]:
tesla.structured_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,Buy/Sell
0,0.966907,0.020257,0.020380,0.017935,0.015529,0.015451,0.013931,0.010360,0.010285,0.002865,...,0.078209,0.099177,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,2
1,0.637550,0.641425,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,...,0.099177,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,2
2,0.641425,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,...,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,0.120783,1
3,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,...,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,0.120783,0.007032,1
4,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,...,0.093011,0.042313,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,1
5,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,...,0.042313,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,0.128688,1
6,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,0.373227,...,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,0.128688,0.073206,1
7,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,0.373227,0.436094,...,0.016872,0.104281,0.189055,0.131329,0.018942,0.002540,0.130902,0.075560,0.000000,1
8,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,0.373227,0.436094,0.283135,...,0.134957,0.216828,0.161078,0.052540,0.036701,0.160666,0.107220,0.034247,0.000000,2
9,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,0.373227,0.436094,0.283135,0.501040,...,0.216828,0.161078,0.052540,0.036701,0.160666,0.107220,0.034247,0.000000,0.083366,0


In [16]:
tesla_lr = ml.svm_ovr('Tesla')

# Note: accuracy is higher in Spyder

In [18]:
acc = 0
mod = svm.SVC()

for i in range(0, 200):
    tesla_ovr.train_model(tesla.structured_df)
    print(acc)
    if tesla_ovr.accuracy > acc:
        acc = tesla_ovr.accuracy
        mod = tesla_ovr.model
tesla_ovr.model = mod
tesla_ovr.accuracy = acc
tesla_ovr.save_current_model()

2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.58
0
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.61
0.5798319327731093
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.60
0.6078431372549019
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.59
0.6078431372549019
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.62
0.6078431372549019
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.63
0.6190476190476191
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.57
0.6330532212885154
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.57
0.6330532212885154
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.62
0.6330532212885154
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accu

The testing accuracy is: 0.65
0.6610644257703081
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.60
0.6610644257703081
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.57
0.6610644257703081
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.58
0.6610644257703081
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.60
0.6610644257703081
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.57
0.6610644257703081
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.59
0.6610644257703081
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.61
0.6610644257703081
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.58
0.6610644257703081
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.61
0.6610644257703081
2    4

The testing accuracy is: 0.61
0.6666666666666666
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.57
0.6666666666666666
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.60
0.6666666666666666
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.60
0.6666666666666666
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.58
0.6666666666666666
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.64
0.6666666666666666
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.62
0.6666666666666666
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.57
0.6666666666666666
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.58
0.6666666666666666
2    475
1    475
0    475
Name: Buy/Sell, dtype: int64
The testing accuracy is: 0.63
0.6666666666666666
2    4